# Field Delineation

In this notebook, we will explore a **method for field delineation** that takes advantage of the kernel functionality within Descartes Labs `Workflows`. Edges in Sentinel-2 imagery are extracted, thresholded, and processed with morphological processing. A region on the map can be selected to output further refined polygons from the underlying raster of the segmented fields.  

You can run the cells in this notebook one at a time by using `Shift-Enter`

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages 
import descarteslabs.workflows as wf
from descarteslabs.workflows import Kernel, conv2d, sqrt

from utils import FieldMap

## Create a Sentinel-2 Composite
Here we use Descartes Labs `Workflows` to create median composite image.

In [ ]:
img_col = wf.ImageCollection.from_id("sentinel-2:L1C", start_datetime="2019-06-01", end_datetime="2019-09-30")
rgb = img_col.pick_bands("red green blue")

nir, red = img_col.unpack_bands("nir red")
ndvi = wf.normalized_difference(nir, red)

composite = rgb.median(axis="images")
composite.visualize("Sentinel-2 Composite")

## Define functions for image processing with kernels

In [ ]:
def dx_kernel(size):
    return Kernel(dims=(size, size), data=[1.0, 0.0, -1.0, 2.0, 0.0, -2.0, 1.0, 0.0, -1.0])

def dy_kernel(size):
    return Kernel(dims=(size, size), data=[1.0, 2.0, 1.0, 0.0, 0.0, 0.0, -1.0, -2.0, -1.0])
    
def dilate_op(map_layer, iters, kernel, layer_name, visualize=False):
    for i in range(iters):
        map_layer = map_layer * 1.0
        map_layer = wf.conv2d(map_layer, kernel) > 0
    if visualize:
        map_layer.visualize(layer_name, scales=[(0, 1)], colormap="plasma")
    return map_layer

def erode_op(map_layer, iters, kernel, layer_name, visualize=False):
    map_layer = ~map_layer
    for i in range(iters):
        map_layer = wf.conv2d(map_layer, kernel) > 0
    map_layer = ~map_layer
    if visualize:
        map_layer.visualize(layer_name, scales=[(0, 1)], colormap="plasma")
    return map_layer

## Extract edges with Sobel kernels

In [ ]:
# Define edge kernels
xder_kernel = dx_kernel(size=3)
yder_kernel = dy_kernel(size=3)

# Make edge images
x_edges = conv2d(composite, xder_kernel)
y_edges = conv2d(composite, yder_kernel)

# Get the gradient magnitude of the edges
mag = sqrt(x_edges**2 + y_edges**2)
mag = mag.max(axis="bands")
mag.visualize("Gradient Magnitude")

# Threshold edges
mag_thresh = mag > 0.08
mag_thresh.visualize("Thresholded Edges", colormap="Greens")

## Use morphological processing (dilation + erosion) to clean up fields

In [ ]:
# Dilate and erode to clean up edges
kernel = wf.Kernel(dims=(3,3), data=[0., 1., 0.,
                                      1., 1., 1.,
                                      0., 1., 0.])

dilated = dilate_op(mag_thresh, iters=2, kernel=kernel, layer_name='dilation', visualize=False)
eroded = erode_op(dilated, iters=2, kernel=kernel, layer_name='erosion', visualize=False)

fields = 1.0*(~eroded)
fields.visualize("Initial Fields", colormap="Greens")

## Get refined field polygons
**Using the draw control on the map below,** draw a rectangle over an area of interest. The fields within the selected region will be cleaned, vectorized, and dispayed on the map.

In [ ]:
# Show the ipyleaflet map
field_map = FieldMap(wf.map)
field_map.fields = fields

field_map.m.center = (36.5698, -120.0924)
field_map.m.zoom = 13
field_map.m

## Access and Export Field Polygons
Field geometries can be accessed as a `GeoDataFrame` by uncommenting the line below.

Alternatively, click the **Export GeoJSON button on the map** to export the vectorized fields into a GeoJSON file that is saved in the working directory.

In [ ]:
#field_map.fields_df.head()